In [136]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [137]:
# encodeamos todas los features categoricos usando One Hot encoding


In [146]:
def normaliza_df(df, ones_hots):
    # inicialmente voy con el tratamiento mas simple sobre los datos faltantes
    df = df.fillna(0)
    df = df.dropna()
    '''
      removemos la columna de la cual el modelo podria aprender que su existencia
      implica el Closed_won (ya que es un valor que se obtiene a posteriori de haber
      ganado y estaria mal usarlo ya que los modelos entrenadas con ella serian 
      incapaces de poder predecir correctamente)
    '''
    df = df.drop(['Sales_Contract_No'], axis=1)
    
    # irrelevantes
    df = df.drop(['Opportunity_Name'], axis=1)

    # dato cte = NaT
    df = df.drop(['Last_Activity'], axis=1)

    # basado en lo charlado con el grupo, ahora subdividimos el df de forma de tener
    # en cuenta los casos cerrados y cobertimos las varaibles categorias en su
    # representacion mas simple (siendo dos casos en Cloased_Won = 1 y Closed:_lost = 0)
    if 'Stage' in df.columns:
        df = df[((df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost'))]
        df['Stage'] = df['Stage'].apply(lambda x: 1 if x == 'Closed Won' else 0)
    
    # hacemos que las variables temporales en las que nos vanos a enfocar sean del tipo correcto
    df['Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'], format="%m/%d/%Y")
    df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'], format="%m/%d/%Y")

    df['Quote_Type'] = df['Quote_Type'].apply(lambda x: 1 if x == 'Binding' else 0)
    
    # todo: convertir estas en una ventana de tiempo
    df = df.drop(['Planned_Delivery_Start_Date'], axis=1)
    df = df.drop(['Planned_Delivery_End_Date'], axis=1)

    categoric_cols = df.columns[df.dtypes==object].tolist() 
    numeric_cols = df.columns[df.dtypes=='float64'].tolist() 
    numeric_cols_2 = df.columns[df.dtypes=='int64'].tolist() 
    date_cols = df.columns[df.dtypes=='datetime64[ns]'].tolist() 
    
    print(categoric_cols)
    print('--------------')
    
    
    def crearOneHotDF(c1):
        if 'Stage' in df.columns:
            onehotencoder = OneHotEncoder(handle_unknown = 'ignore')
            onehotencoder.fit(df[[c1]])
            ones_hots[c1] = onehotencoder
        else:
            onehotencoder = ones_hots[c1]
        
        M = pd.DataFrame(onehotencoder.transform(df[[c1]]).toarray())
            
        num_cols = len(list(M))
        rng = range(0, num_cols)
        new_cols = [c1 + str(i) for i in rng] 
        M.columns = new_cols[:num_cols]
        return M

    categorical = pd.concat([crearOneHotDF(i) for i in categoric_cols], axis=1)

    df = pd.concat([categorical, df[numeric_cols], df[numeric_cols_2], df[date_cols]], axis=1)
    
    df = df.sort_values(by="Opportunity_Created_Date")
    
    df = df.drop(columns = 'Opportunity_Created_Date')
    df = df.drop(columns = 'Account_Created_Date')

    df = df.fillna(0)
    df = df.dropna()
    
    return df

In [139]:
df_train = pd.read_csv('/usr/src/tp2-1/Train_TP2_Datos_2020-2C.csv')


In [147]:
ones_hots = {}
ndf = normaliza_df(df_train, ones_hots)

['Region', 'Territory', 'Bureaucratic_Code', 'Source ', 'Billing_Country', 'Account_Name', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Delivery_Terms', 'Brand', 'Product_Type', 'Size', 'Product_Category_B', 'Price', 'Currency', 'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP_(converted)_Currency', 'Month', 'Delivery_Quarter', 'Actual_Delivery_Date', 'Total_Amount_Currency', 'Total_Taxable_Amount_Currency', 'Prod_Category_A']
--------------


In [149]:
ndf.shape

(16947, 4517)

In [154]:
ndf.to_csv('/usr/src/tp2-1/210215_tp2_train_feng.csv')

In [150]:
df_test = pd.read_csv('/usr/src/tp2-1/Test_TP2_Datos_2020-2C.csv')

In [151]:
ndft = normaliza_df(df_test, ones_hots)

['Region', 'Territory', 'Bureaucratic_Code', 'Source ', 'Billing_Country', 'Account_Name', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Delivery_Terms', 'Brand', 'Product_Type', 'Size', 'Product_Category_B', 'Price', 'Currency', 'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP_(converted)_Currency', 'Month', 'Delivery_Quarter', 'Actual_Delivery_Date', 'Total_Amount_Currency', 'Total_Taxable_Amount_Currency', 'Prod_Category_A']
--------------


In [152]:
ndft.shape

(2551, 4516)

In [153]:
ndft.to_csv('/usr/src/tp2-1/210215_tp2_test_feng.csv')